In [130]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize
from clapton.ansatzes import * 
from qiskit_research.utils.convenience import *

In [131]:
def circuit_to_tableau(circuit: stim.Circuit) -> stim.Tableau:
    s = stim.TableauSimulator()
    s.do_circuit(circuit)
    return s.current_inverse_tableau() ** -1

In [132]:
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
coeffs = np.random.random(len(paulis))
from clapton.depolarization import GateGeneralDepolarizationModel


## Altering the Ansatz itself

In [133]:
rng = np.random.default_rng()

TWIRL_GATES_CX = (
        (('I', 'I'), ('I', 'I')),
        (('I', 'X'), ('I', 'X')),
        (('I', 'Y'), ('Z', 'Y')),
        (('I', 'Z'), ('Z', 'Z')),
        (('X', 'I'), ('X', 'X')),
        (('X', 'X'), ('X', 'I')),
        (('X', 'Y'), ('Y', 'Z')),
        (('X', 'Z'), ('Y', 'Y')),
        (('Y', 'I'), ('Y', 'X')),
        (('Y', 'X'), ('Y', 'I')),
        (('Y', 'Y'), ('X', 'Z')),
        (('Y', 'Z'), ('X', 'Y')),
        (('Z', 'I'), ('Z', 'I')),
        (('Z', 'X'), ('Z', 'X')),
        (('Z', 'Y'), ('I', 'Y')),
        (('Z', 'Z'), ('I', 'Z')),
    )

pauli_twirl_dict = {"I":0,"X":1,"Y":2,"Z":3}

def twirled_circular_ansatz(N, reps=1, fix_2q=False):
    pcirc = ParametrizedCliffordCircuit()
    for _ in range(reps):
        for i in range(N):
            pcirc.RY(i)
        for i in range(N):
            pcirc.RZ(i)
        for i in range(N):
            control = (i-1) % N
            target = i
            if fix_2q:
                
                (before0, before1), (after0, after1) = TWIRL_GATES_CX[
                    rng.integers(len(TWIRL_GATES_CX))]

                pcirc.PauliTwirl(control).fix(pauli_twirl_dict[before0])
                pcirc.PauliTwirl(target).fix(pauli_twirl_dict[before1])
                pcirc.Q2(control, target).fix(1)
                pcirc.PauliTwirl(control).fix(pauli_twirl_dict[after0])
                pcirc.PauliTwirl(target).fix(pauli_twirl_dict[after1])
            else:
                pcirc.Q2(control, target)
    for i in range(N):
        pcirc.RY(i)
    for i in range(N):
        pcirc.RZ(i)

    
    return pcirc



In [134]:
# let's add a noise model where we specify global 1q and 2q gate errors


nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.02)
# nm = None

pauli_twirl = True


if pauli_twirl:
    twirl_ansatz = twirled_circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)
    twirl_ansatz.add_depolarization_model(nm)

    pauli_twirl_list = [twirled_circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True) for _ in range(100)]
    pauli_twirl_list = [circuit.add_depolarization_model(nm) for circuit in pauli_twirl_list]

    twirl_ansatz.add_pauli_twirl_list(pauli_twirl_list)

else: 
    twirl_ansatz = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)
    twirl_ansatz.add_depolarization_model(nm)

### Check if the circuits are the same

In [135]:
circ_base = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)
twirled_circ = twirled_circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)

circuit_to_tableau(circ_base.stim_circuit())==circuit_to_tableau(twirled_circ.stim_circuit())


True

In [136]:
circuit_to_tableau(circ_base.stim_circuit()),circuit_to_tableau(twirled_circ.stim_circuit())

(stim.Tableau.from_conjugated_generators(
     xs=[
         stim.PauliString("+XXX"),
         stim.PauliString("+_XX"),
         stim.PauliString("+XX_"),
     ],
     zs=[
         stim.PauliString("+ZZZ"),
         stim.PauliString("+ZZ_"),
         stim.PauliString("+_ZZ"),
     ],
 ),
 stim.Tableau.from_conjugated_generators(
     xs=[
         stim.PauliString("+XXX"),
         stim.PauliString("+_XX"),
         stim.PauliString("+XX_"),
     ],
     zs=[
         stim.PauliString("+ZZZ"),
         stim.PauliString("+ZZ_"),
         stim.PauliString("+_ZZ"),
     ],
 ))

In [123]:
twirl_ansatz.number_parametrized_gates()

12

In [124]:
twirl_ansatz.read()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [125]:
twirl_ansatz.stim_circuit().diagram()

q0: -I-I-I-X-X-I-@-Z-----I-I---
           |     |
q1: -I-I---|---Y-X-Y-Z-@-Z-I-I-
           |           |
q2: -I-I-Y-@-Y-------X-X-X-I-I-

In [126]:
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    twirl_ansatz,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20,                        # budget per genetic algorithm instance
)

STARTING ROUND 0




started GA at id 1 with 1 procs
started GA at id 2 with 1 procs

started GA at id None with 1 procs


started GA at id 3 with 1 procs



/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
/home/dbharadwaj/anaconda3/envs/clapton/lib/python3.10/site-packages/pygad/pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evo

[0, array([3.09837598, 1.54918799, 1.54918799, 0.        ]), array([1, 0, 1, 3, 3, 2, 2, 0, 1, 0, 0, 0], dtype=object)]
[0, array([3.09837598, 1.54918799, 1.54918799, 0.        ]), array([3, 0, 1, 2, 2, 2, 0, 1, 3, 0, 3, 0], dtype=object)]

[0, array([3.09837598, 1.54918799, 1.54918799, 0.        ]), array([2, 1, 3, 3, 1, 2, 2, 2, 1, 3, 3, 1], dtype=object)][0, array([3.09837598, 1.54918799, 1.54918799, 0.        ]), array([3, 0, 1, 0, 2, 1, 3, 3, 2, 2, 1, 2], dtype=object)]
[1, array([3.09837598, 1.54918799, 1.54918799, 0.        ]), array([1, 2, 3, 0, 0, 3, 2, 2, 2, 2, 2, 1], dtype=object)]
[1, array([3.09837598, 1.54918799, 1.54918799, 0.        ]), array([1, 2, 0, 0, 1, 3, 3, 0, 1, 1, 1, 2], dtype=object)]
[1, array([3.09837598, 1.54918799, 1.54918799, 0.        ]), array([1, 0, 3, 0, 1, 1, 0, 3, 1, 0, 1, 0], dtype=object)]
[1, array([3.09837598, 1.54918799, 1.54918799, 0.        ]), array([3, 1, 3, 3, 2, 0, 0, 0, 3, 1, 0, 0], dtype=object)]
[2, array([3.09837598, 1.54918799, 1.549

In [127]:
ks_best

[1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 1]

In [128]:
energy_noisy, energy_noiseless

(1.549187990124281, 1.549187990124281)

In [129]:
# differrence
np.abs(energy_noisy-energy_noiseless)

0.0

In [70]:
# the corresponding circuit is
twirl_ansatz.assign(ks_best)
twirl_ansatz.snapshot_noiseless().circ_snapshot_noiseless.diagram()

q0: -SQRT_Y-S-----X-@-SQRT_Y_DAG-Z-------
                  | |
q1: -SQRT_Y-I-----|-X-@----------I-Z-----
                  |   |
q2: -I------S_DAG-@---X----------I-S_DAG-